In [2]:
import os
import boto3
import pandas as pd

DB_ACCESS_KEY = pd.read_csv('DB_ACCESS_KEY.csv')
aws_access_key_id = DB_ACCESS_KEY['Access key ID'][0]
aws_secret_access_key = DB_ACCESS_KEY['Secret access key'][0]
rds_key_pass = pd.read_csv('db_key_pass.csv')
username = rds_key_pass['ID'][0]
password = rds_key_pass['PASS'][0]
url = rds_key_pass['URL'][0]

In [3]:
def getPastStockPrices(refresh_counter) -> pd.DataFrame:
    """
    returns a pandas dataframe structured as follows:
    company name, ticker, sentiment score, sentiment magnitude, sentiment score change, sentiment magnitude change
    """
    # # create a boto3 client and import all stock prices from it
    dynamodb = boto3.resource('dynamodb', region_name='us-east-2', aws_access_key_id=aws_access_key_id, aws_secret_access_key=aws_secret_access_key)
    table = dynamodb.Table('StockPrices')
    # keep scanning until we have all the data in the table
    # keep scanning until we have all the data in the table
    response = table.scan()
    data = response['Items']
    while 'LastEvaluatedKey' in response:
        response = table.scan(ExclusiveStartKey=response['LastEvaluatedKey'])
        data.extend(response['Items'])
    # convert the data to a pandas dataframe and drop the stock column
    stock_prices = pd.DataFrame(data)
    # stock_prices.set_index('Date', inplace=True)
    # convert Date column to datetime
    stock_prices['Date'] = pd.to_datetime(stock_prices['Date'])
    # make the index the Date column
    # stock_prices.sort_index(ascending=False, inplace=True)
    return stock_prices


In [4]:
stock_prices = getPastStockPrices(0)
stock_prices

,Date,100_day_MA,split,4_week_high,10_week_low,200_day_MA,open,Stock,volume,dividend,50_day_MA,10_week_high,close,low,4_week_low,52_week_low,adj_close,high,30_day_MA,52_week_high
0,2023-02-28,7.5760000000000005,1.0,11.31,4.9,6.276125,8.86,MOMO,1774239,0.0,10.127999999999998,11.31,8.81,8.7,8.81,4.21,8.81,8.902,10.127999999999998,12.3891608391608
1,2023-03-01,7.615399999999999,1.0,11.31,4.9,6.297375000000001,9.22,MOMO,1198583,0.0,10.087666666666667,11.31,9.01,8.99,8.81,4.21,9.01,9.4,10.087666666666667,12.3891608391608
2,2023-03-02,7.6561,1.0,11.31,4.9,6.317874999999999,8.91,MOMO,1250331,0.0,10.063333333333333,11.31,9.16,8.8642,8.81,4.21,9.16,9.26,10.063333333333333,12.3891608391608
3,2023-03-03,7.699300000000001,1.0,11.31,4.9,6.339425,9.12,MOMO,942348,0.0,10.039,11.31,9.3,9.08,8.81,4.21,9.3,9.37,10.039,12.3891608391608
4,2023-03-06,7.7413,1.0,11.31,4.9,6.358924999999999,9.25,MOMO,1100510,0.0,9.994333333333334,11.31,8.95,8.9,8.81,4.21,8.95,9.29,9.994333333333334,12.3891608391608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554285,2023-03-16,24.1168436259868,1.0,25.0,22.925260515077,24.033158742976102,24.78,GLOG-P-A,437,0.0,24.739796666666667,25.0,24.504,24.504,24.25,22.5153045844319,24.504,24.78,24.739796666666667,25.0
554286,2023-03-17,24.124794007764976,1.0,25.0,22.925260515077,24.03707013207972,24.79,GLOG-P-A,6207,0.0,24.715053333333334,25.0,24.1,24.0,24.1,22.5153045844319,24.1,25.07,24.715053333333334,25.0
554287,2023-03-20,24.122729493723305,1.0,25.0,22.925260515077,24.036778250085504,24.2,GLOG-P-A,9468,0.0,24.663973333333335,25.0,23.25,22.4,23.25,22.5153045844319,23.25,24.2202,24.663973333333335,25.0
554288,2023-03-21,24.127176303610717,1.0,25.0,22.925260515077,24.03853777212424,23.68,GLOG-P-A,3572,0.0,24.64130666666667,25.0,23.95,23.6,23.25,22.5153045844319,23.95,23.9999,24.64130666666667,25.0


In [5]:
final_stock = stock_prices.copy()

In [6]:
final_stock['Date'] = final_stock.index

In [7]:
final_stock

,Date,100_day_MA,split,4_week_high,10_week_low,200_day_MA,open,Stock,volume,dividend,50_day_MA,10_week_high,close,low,4_week_low,52_week_low,adj_close,high,30_day_MA,52_week_high
0,0,7.5760000000000005,1.0,11.31,4.9,6.276125,8.86,MOMO,1774239,0.0,10.127999999999998,11.31,8.81,8.7,8.81,4.21,8.81,8.902,10.127999999999998,12.3891608391608
1,1,7.615399999999999,1.0,11.31,4.9,6.297375000000001,9.22,MOMO,1198583,0.0,10.087666666666667,11.31,9.01,8.99,8.81,4.21,9.01,9.4,10.087666666666667,12.3891608391608
2,2,7.6561,1.0,11.31,4.9,6.317874999999999,8.91,MOMO,1250331,0.0,10.063333333333333,11.31,9.16,8.8642,8.81,4.21,9.16,9.26,10.063333333333333,12.3891608391608
3,3,7.699300000000001,1.0,11.31,4.9,6.339425,9.12,MOMO,942348,0.0,10.039,11.31,9.3,9.08,8.81,4.21,9.3,9.37,10.039,12.3891608391608
4,4,7.7413,1.0,11.31,4.9,6.358924999999999,9.25,MOMO,1100510,0.0,9.994333333333334,11.31,8.95,8.9,8.81,4.21,8.95,9.29,9.994333333333334,12.3891608391608
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
554285,554285,24.1168436259868,1.0,25.0,22.925260515077,24.033158742976102,24.78,GLOG-P-A,437,0.0,24.739796666666667,25.0,24.504,24.504,24.25,22.5153045844319,24.504,24.78,24.739796666666667,25.0
554286,554286,24.124794007764976,1.0,25.0,22.925260515077,24.03707013207972,24.79,GLOG-P-A,6207,0.0,24.715053333333334,25.0,24.1,24.0,24.1,22.5153045844319,24.1,25.07,24.715053333333334,25.0
554287,554287,24.122729493723305,1.0,25.0,22.925260515077,24.036778250085504,24.2,GLOG-P-A,9468,0.0,24.663973333333335,25.0,23.25,22.4,23.25,22.5153045844319,23.25,24.2202,24.663973333333335,25.0
554288,554288,24.127176303610717,1.0,25.0,22.925260515077,24.03853777212424,23.68,GLOG-P-A,3572,0.0,24.64130666666667,25.0,23.95,23.6,23.25,22.5153045844319,23.95,23.9999,24.64130666666667,25.0


In [8]:
import pymysql
from sqlalchemy import create_engine
import sqlalchemy

# Establish connection to the MySQL database
conn = pymysql.connect(host=url, user=username, password=password, db='stock_data')

# Create a SQLAlchemy engine object
engine = create_engine(f'mysql+pymysql://{username}:{password}@{url}/stock_data', echo=False)

# Convert the pandas DataFrame to a MySQL table
stock_prices.to_sql(name='Prices', con=engine, if_exists='replace', index=False, dtype={'Date': sqlalchemy.types.DATETIME})

# Close the connection
conn.close()